In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import math
import random
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from collections import Counter

In [ ]:
class Fairness_Goodness():
    def __init__(self, G):
        fairness, goodness = self.initialize_scores(G)

        nodes = G.nodes()
        iter = 0
        while iter < 100:
            df = 0
            dg = 0

            # print('-----------------')
            # print("Iteration number", iter)

            # print('Updating goodness')
            for node in nodes:
                inedges = G.in_edges(node, data='weight')
                g = 0
                for edge in inedges:
                    g += fairness[edge[0]] * edge[2]

                try:
                    dg += abs(g / len(inedges) - goodness[node])
                    goodness[node] = g / len(inedges)
                except:
                    pass

            # print('Updating fairness')
            for node in nodes:
                outedges = G.out_edges(node, data='weight')
                f = 0
                for edge in outedges:
                    f += 1.0 - abs(edge[2] - goodness[edge[1]]) / 2.0

                try:
                    df += abs(f / len(outedges) - fairness[node])
                    fairness[node] = f / len(outedges)
                except:
                    pass

            # print('Differences in fairness score = %.6f and goodness score = %.6f' % (df, dg))
            if df < math.pow(10, -6) and dg < math.pow(10, -6):
                break
            iter += 1

        self.fairness = fairness
        self.goodness = goodness

    def initialize_scores(self, G):
        fairness = {}
        goodness = {}

        nodes = G.nodes()
        for node in nodes:
            fairness[node] = 1
            try:
                goodness[node] = G.in_degree(node, weight='weight') * 1.0 / G.in_degree(node)
            except:
                goodness[node] = 0
        return fairness, goodness

    def compute_fairness_goodness(self, G):
        return self.fairness, self.goodness

    def predict_weight(self, u, v):
        pred_w = self.fairness[u] * self.goodness[v]
        return pred_w

In [ ]:
class Reciprocal():
    def __init__(self, G):
        self.G = G

    def predict_weight(self, u, v):
        if self.G.has_edge(v, u):
            return self.G[v][u]['weight']
        else:
            return 0

In [ ]:
class Bias_Deserve():

    def __init__(self, G):

        bias, des = self.initialize_scores(G)

        nodes = G.nodes()
        iter = 0
        while iter < 100:
            db = 0
            dd = 0

            # print('-----------------')
            # print("Iteration number", iter)

            # print('Updating des')
            for node in nodes:
                inedges = G.in_edges(node, data='weight')
                d = 0
                for edge in inedges:
                    X = max(0, bias[edge[0]] * edge[2])
                    d += edge[2] * (1 - X)
                try:
                    dd += abs(d / len(inedges) - des[node])
                    des[node] = d / len(inedges)
                except:
                    pass

            # print('Updating bias')
            for node in nodes:
                outedges = G.out_edges(node, data='weight')
                b = 0
                for edge in outedges:
                    b += (edge[2] - des[edge[1]]) / 2.0
                try:
                    db += abs(b / len(outedges) - bias[node])
                    bias[node] = b / len(outedges)
                except:
                    pass

            # print('Differences in bias score = %.6f and des score = %.6f' % (db, dd))
            if db < math.pow(10, -6) and dd < math.pow(10, -6):
                break
            iter += 1

        self.bias = bias
        self.des = des

    def initialize_scores(self, G):
        des = {}
        bias = {}
        nodes = G.nodes()
        for node in nodes:
            bias[node] = 1
            try:
                des[node] = G.in_degree(node, weight='weight') * (1.0 - max(0, bias[node] * G.in_degree(node, weight='weight'))) / G.in_degree(node)
            except:
                des[node] = 0
        return bias, des
    
    def compute_bias_des(self, G):
        return self.bias, self.des

    def predict_weight(self, u, v):
        return self.des[v]

In [ ]:
def signed_hits(G, max_iter=100, tol=1.0e-8, normalized=True):

    if type(G) == nx.MultiGraph or type(G) == nx.MultiDiGraph:
        raise Exception("hits() not defined for graphs with multiedges.")
    if len(G) == 0:
        return {}, {}
    # choose fixed starting vector if not given

    h_p = dict.fromkeys(G, 1.0 / G.number_of_nodes())
    h_n = dict.fromkeys(G, - 1.0 / G.number_of_nodes())
    h = h_p
    a = h_p
    for _ in range(max_iter):  # power iteration: make up to max_iter iterations
        h_p_last = h_p
        h_n_last = h_n

        h_p = dict.fromkeys(h_p_last.keys(), 0)
        h_n = dict.fromkeys(h_n_last.keys(), 0)
        a_p = dict.fromkeys(h_p_last.keys(), 0)
        a_n = dict.fromkeys(h_n_last.keys(), 0)

        # this "matrix multiply" looks odd because it is
        # doing a left multiply a^T=hlast^T*G

        for u in h_p:
            for v in G.pred[u]:
                if G[v][u]['weight'] >= 0 :
                    a_p[u] += h_p_last[v] * G[v][u]['weight']
                else :
                    a_n[u] -= h_n_last[v] * G[v][u]['weight']
        for u in h_p:
            for v in G.succ[u]:
                if G[u][v]['weight'] >= 0:
                    h_p[u] += a_p[v] * G[u][v]['weight']
                else :
                    h_n[u] -= a_n[v] * G[u][v]['weight']


        # normalize vector
        s = 1.0 / max(h_p.values())
        for n in h_p:
            h_p[n] *= s
        # normalize vector
        s = -1.0 / min(h_n.values())
        for n in h_n:
            h_n[n] *= s
        # normalize vector
        s = 1.0 / max(a_p.values())
        for n in a_p:
            a_p[n] *= s
        # normalize vector
        s = -1.0 / min(a_n.values())
        for n in a_n:
            a_n[n] *= s

        for key, value in h.items():
            h[key] = h_p[key] - h_n[key]
            a[key] = a_p[key] - a_n[key]

        # check convergence, l1 norm
        err = sum([abs(h_p[n] - h_p_last[n]) for n in h_p] + [abs(h_n[n] - h_n_last[n]) for n in h_n] )
        if err < tol:
            break
    else:
        raise nx.PowerIterationFailedConvergence(max_iter)
    if normalized:
        s = 1.0 / sum(a.values())
        for n in a:
            a[n] *= s
        s = 1.0 / sum(h.values())
        for n in h:
            h[n] *= s
    return h, a

class Signed_Hits():
    def __init__(self, G):
        self.G = G
        h, a = signed_hits(G, max_iter=500)
        G_hits = self.G.copy()
        w_in = {}
        w_out = {}
        Hits_total_out = {}
        Hits_total_in = {}

        for (u, w) in G_hits.nodes(data='weight'):
            w_in_v = 0
            w_out_v = 0
            Hits_total_out_v = 0
            Hits_total_in_v = 0
            for edge in G_hits.out_edges(u, data='weight'):
                w_out_v += edge[2] * h[edge[0]]
                Hits_total_out_v += h[edge[0]]
            for edge in G_hits.in_edges(u, data='weight'):
                w_in_v += edge[2] * a[edge[1]]
                Hits_total_in_v += a[edge[1]]
            w_in[u] = w_in_v
            w_out[u] = w_out_v
            Hits_total_in[u] = Hits_total_in_v
            Hits_total_out[u] = Hits_total_out_v
        nx.set_node_attributes(G_hits, w_in, 'w_in')
        nx.set_node_attributes(G_hits, w_out, 'w_out')
        nx.set_node_attributes(G_hits, Hits_total_in, 'Hits_total_in')
        nx.set_node_attributes(G_hits, Hits_total_out, 'Hits_total_out')

        self.G_hits = G_hits

    def predict_weight(self, u, v):

        pred_w = 0
        if self.G_hits.nodes[u]['Hits_total_out'] != 0:
            pred_w += self.G_hits.nodes[u]['w_out'] / self.G_hits.nodes[u]['Hits_total_out']
        if self.G_hits.nodes[v]['Hits_total_in'] != 0:
            pred_w += self.G_hits.nodes[v]['w_in'] / self.G_hits.nodes[v]['Hits_total_in']
        pred_w /= 2
        return pred_w

In [ ]:
def sample_edges(G, num_edges):
    all_edges = list(nx.edges(G))
    return random.sample(all_edges, num_edges)

In [ ]:
def read_graph(filename):
    G = nx.DiGraph()
    f = open(f'/content/gdrive/MyDrive/{filename}', "r")
    for l in f:
        ls = l.strip().split(",")
        G.add_edge(ls[0], ls[1], weight = float(ls[2])/10.0) # the weight should already be in the range of -1 to 1
    f.close()
    return G

def run_prediction_algorithms(G, num_edges_to_test):
    algorithms = ['Fairness_Goodness', 'Reciprocal', 'Bias_Deserve', 'Signed_Hits']
    algorithm_type = ['fg', 'r', 'bd', 'sh']
    G_copy = G.copy()
    test_edges = sample_edges(G, num_edges_to_test)
    true_weights = []
    predicted_weights = {'fg': [], 'r': [], 'bd': [], 'sh': []}
    rmse = {}
    pcc = {}

    for (u,v) in test_edges:
        true_weights.append(G[u][v]['weight'])
        G_copy.remove_edge(u, v)

        for key in algorithm_type:
            if key == 'fg':
                method = Fairness_Goodness(G_copy)
            elif key == 'r':
                method = Reciprocal(G_copy)
            elif key == 'bd':
                method = Bias_Deserve(G_copy)
            else:
                method = Signed_Hits(G_copy)

            predicted_weights[key].append(method.predict_weight(u, v))

        G_copy.add_edge(u, v, weight=G[u][v]['weight'])

    for t in algorithm_type:
        rmse[t] = mean_squared_error(true_weights, predicted_weights[t], squared=False)
        pcc[t] = pearsonr(true_weights, predicted_weights[t])[0]

    rmse_stack = np.vstack(([rmse[x] for x in algorithm_type]))
    pcc_stack = np.vstack(([pcc[x] for x in algorithm_type]))
    df_rmse = pd.DataFrame(rmse_stack, index=algorithms, columns=['Value'])
    df_pcc = pd.DataFrame(pcc_stack, index=algorithms, columns=['Value'])
    print(df_rmse)
    print(df_pcc)

**Bitcoin OTC**

In [ ]:
G_otc = read_graph('soc-sign-bitcoinotc.csv')
run_prediction_algorithms(G_otc, 1000)

                      Value
Fairness_Goodness  0.330024
Reciprocal         0.315943
Bias_Deserve       0.338862
Signed_Hits        0.299825
                      Value
Fairness_Goodness  0.388601
Reciprocal         0.511097
Bias_Deserve       0.369605
Signed_Hits        0.542348


**Bitcoin Alpha**

In [ ]:
G_alpha = read_graph('soc-sign-bitcoinalpha.csv')
run_prediction_algorithms(G_alpha, 1000)

                      Value
Fairness_Goodness  0.294868
Reciprocal         0.263249
Bias_Deserve       0.299613
Signed_Hits        0.288076
                      Value
Fairness_Goodness  0.268004
Reciprocal         0.519646
Bias_Deserve       0.262165
Signed_Hits        0.253705
